In [1]:
from underthesea import text_normalize, word_tokenize
from LLM import Process_LLM
import requests
import csv
import re

stopwords = []
with requests.get("https://raw.githubusercontent.com/OneLevelStudio/stopwords/refs/heads/main/vi.txt") as r:
    if r.status_code != 200:
        raise ValueError("Error:", r.status_code, r.text)
    else:
        stopwords = r.text.split("\n")
specialchars = ['!', '"', '#', '$', '%', '&', "'", '(', ')', '*', '+', ',', '-', '.', '..', '...', '/', ':', ';', '<', '=', '>', '?', '@', '[', '\\', ']', '^', '_', '`', '{', '|', '}', '~']

def is_number(s):
    return bool(re.fullmatch(r'[\d,. ]+', s))

def extract_keywords(s):
    s = text_normalize(s)
    words = word_tokenize(s)
    words = [w.lower() for w in words]
    words = list(set(words))
    words = [w for w in words if not is_number(w)]
    # words = [w for w in words if w not in stopwords]
    words = [w for w in words if w not in specialchars]
    return words

with open('url/cache', mode='r', newline='', encoding='utf-8') as f:
    csv_reader = csv.DictReader(f)
    thutuchanhchinhs = [row for row in csv_reader]
    thutuchanhchinhs = sorted(thutuchanhchinhs, key=lambda item: len(item["Tên thủ tục"]))
    for i in range(len(thutuchanhchinhs)):
        thutuchanhchinhs[i]['keywords'] = extract_keywords(thutuchanhchinhs[i]['Tên thủ tục'] + "\n" + thutuchanhchinhs[i]['Lĩnh vực'])

In [2]:
test_str = [
"tôi sắp lập gia đình",
"tôi sắp lập gia đình thì cần làm gì",
"sắp khởi nghiệp thì cần thủ tục gì",
"khởi nghiệp cần giấy tờ gì",
"giấy chứng nhận đăng ký doanh nghiệp",
"kết hôn thủ tục nào",
"giấy tờ liên quan đến trung học phổ thông",
"sinh con thủ tục nào",
"đất đai",
"thủ tục xây nhà cấp 3, 4",
"xây dựng công trình cấp 3, 4",
"phúc khảo bài thi thpt",
]

In [3]:
question = test_str[0]

prompt = f"""\
Tôi có câu hỏi "{question}".
Tôi muốn tìm kiếm thủ tục hành chính liên quan, giúp tôi liệt kê 1 từ khoá quan trọng nhất để tìm kiếm, không trả lời dài dòng.
"""

res = Process_LLM(prompt, vendor="openrouter")
q_keywords = extract_keywords(res)
print(question)
print(res)
print(q_keywords)


thutuc_counts = []
max_count = -1
for thutuc in thutuchanhchinhs:
    count = 0
    for k in q_keywords:
        if k in thutuc["keywords"]:
            count += 1
    if count > max_count:
        max_count = count
    thutuc_counts.append(count)

for idx, count in enumerate(thutuc_counts):
    if count == max_count:
        print(thutuchanhchinhs[idx]["Tên thủ tục"])

tôi sắp lập gia đình
**Đăng ký kết hôn**

['đăng ký', 'kết hôn']
Thủ tục đăng ký kết hôn
Thủ tục đăng ký lại kết hôn
Thủ tục đăng ký kết hôn lưu động
Thủ tục đăng ký kết hôn có yếu tố nước ngoài
Thủ tục đăng ký lại kết hôn có yếu tố nước ngoài
